In [1]:
from fbref_get import *

In [2]:
players_22 = get_outfield_data(2022)
print('22 done')
players_21 = get_outfield_data(2021)
print('21 done')
players_20 = get_outfield_data(2020)
print('20 done')
players_19 = get_outfield_data(2019)
print('19 done')

22 done
21 done
20 done
19 done


In [3]:
wc_18 = pd.read_csv('fbref/wc_2018.csv', header=1)
wc_14 = pd.read_csv('fbref/wc_2014.csv', header=1)

fifa_data = pd.read_csv('fifa/fifa_22.csv')

/Users/tomas/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
players_22.columns

Index(['season', 'player', 'nationality', 'position', 'squad', 'age',
       'birth_year', 'games', 'games_starts', 'minutes',
       ...
       'on_goals_for', 'on_goals_against', 'plus_minus', 'plus_minus_per90',
       'plus_minus_wowy', 'on_xg_for', 'on_xg_against', 'xg_plus_minus',
       'xg_plus_minus_per90', 'xg_plus_minus_wowy'],
      dtype='object', length=170)

In [10]:
players_22 = players_22.dropna()
players_21 = players_21.dropna()
players_20 = players_20.dropna()
players_19 = players_19.dropna()

In [11]:
players_22['age'] = np.array([name[:-4] for name in players_22.age])

/Users/tomas/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
# players_22 = players_22[stats]
# players_21 = players_21[stats]
# players_20 = players_20[stats]
# players_19 = players_19[stats]

# wc_18 = wc_18[stats]
# wc_14 = wc_14[stats]

In [13]:
players_22['season'] = [2022] * len(players_22.player)
players_21['season'] = [2021] * len(players_21.player)
players_20['season'] = [2020] * len(players_20.player)
players_19['season'] = [2019] * len(players_19.player)

wc_18['season'] = [2018] * len(wc_18.Player)
wc_14['season'] = [2014] * len(wc_14.Player)

/Users/tomas/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
# players_22['player'] = np.array([name[:-9] for name in players_22.player])
# players_21['player'] = np.array([name[:-9] for name in players_21.player])
# players_20['player'] = np.array([name[:-9] for name in players_20.player])
# players_19['player'] = np.array([name[:-9] for name in players_19.player])

wc_18['Player'] = np.array([name[:-9] for name in wc_18.Player])
wc_14['Player'] = np.array([name[:-9] for name in wc_14.Player])

In [15]:
players = pd.concat([players_19, players_20, players_21, players_22], axis=0)

wc = pd.concat([wc_14, wc_18], axis=0)

In [16]:
player_names = np.unique(players.player)

In [17]:
sum_data = {}

for column_name in players.columns: 
    sum_data[column_name] = []

    
for player_name in player_names:

    df_player = players[players.player == player_name]

    for column_name in df_player.columns:
        
        if column_name == 'season': 
            sum_data[column_name].append(list(df_player.season)[-1])
        elif column_name == 'position':
            sum_data[column_name].append(list(df_player.position)[-1])
        elif column_name == 'squad':
            sum_data[column_name].append(list(df_player.squad)[-1])
        elif column_name == 'age':
            sum_data[column_name].append(int(list(df_player.age)[-1]))
        elif column_name == 'player':
            sum_data[column_name].append(list(df_player.player)[-1])
        elif column_name == 'nationality':
            sum_data[column_name].append(list(df_player.nationality)[-1])
        elif column_name == 'birth_year':
            sum_data[column_name].append(list(df_player.birth_year)[-1])
        else:
            sum_data[column_name].append(np.sum(df_player[column_name]))
            

In [18]:
df_sum = pd.DataFrame.from_dict(sum_data) # sum of domestic leagues

In [19]:
wc_data = {
    'wc_starts': [],
    'wc_games': [],
    'wc_minutes': [],
    'wc_goals': [],
    'wc_assists': [],
    'wc_cards_yellow': [],
    'wc_cards_red': [],
}

player_names = np.unique(df_sum.player)

non_wc_players = 0

for player_name in player_names:

    df_player = wc[wc.Player == player_name]
    
    if len(df_player.Player) == 0:
        non_wc_players += 1
        
        wc_data['wc_starts'].append(0)
        wc_data['wc_games'].append(0)
        wc_data['wc_minutes'].append(0)
        wc_data['wc_goals'].append(0)
        wc_data['wc_assists'].append(0)
        wc_data['wc_cards_yellow'].append(0)
        wc_data['wc_cards_red'].append(0)
    
    else:
        sum_stars = len(df_player.Player)
        sum_games = np.sum(df_player.Starts)
        sum_minutes = np.sum(df_player.Min)
        sum_goals = np.sum(df_player.Gls)
        sum_assists = np.sum(df_player.Ast)
        sum_yellow = np.sum(df_player.CrdY)
        sum_red = np.sum(df_player.CrdR)

        wc_data['wc_starts'].append(sum_stars)
        wc_data['wc_games'].append(sum_games)
        wc_data['wc_minutes'].append(sum_minutes)
        wc_data['wc_goals'].append(sum_goals)
        wc_data['wc_assists'].append(sum_assists)
        wc_data['wc_cards_yellow'].append(sum_yellow)
        wc_data['wc_cards_red'].append(sum_red)
        
print('Non WC players:', non_wc_players)
print('Total players:', len(wc_data['wc_starts']))

Non WC players: 3992
Total players: 4486


In [20]:
df_sum['wc_starts'] = wc_data['wc_starts']
df_sum['wc_games'] = wc_data['wc_games']
df_sum['wc_minutes'] = wc_data['wc_minutes']
df_sum['wc_goals'] = wc_data['wc_goals']
df_sum['wc_assists'] = wc_data['wc_assists']
df_sum['wc_cards_yellow'] = wc_data['wc_cards_yellow']
df_sum['wc_cards_red'] = wc_data['wc_cards_red']

In [21]:
team_names = [
    # La Liga
    ('Real Madrid','Real Madrid CF'), 
    ('Atlético Madrid','Atlético de Madrid'), 
    ('Real Sociedad','Real Sociedad'), 
    ('Sevilla','Sevilla FC'), 
    ('Betis','Real Betis Balompié'), 
    ('Rayo Vallecano','Rayo Vallecano'),
    ('Barcelona','FC Barcelona'),
    ('Athletic Club','Athletic Club de Bilbao'),
    ('Espanyol','RCD Espanyol de Barcelona'),
    ('Osasuna','CA Osasuna'),
    ('Valencia','Valencia CF'),
    ('Villarreal','Villarreal CF'),
    ('Celta Vigo','RC Celta de Vigo'),
    ('Mallorca','RCD Mallorca'),
    ('Alavés','Deportivo Alavés'),
    ('Granada','Granada CF'),
    ('Elche','Elche CF'),
    ('Cádiz','Cádiz CF'),
    ('Getafe','Getafe CF'),
    ('Levante','Levante Unión Deportiva'),
    # Bundesliga
    ('Bayern Munich','FC Bayern München'),
    ('Dortmund','Borussia Dortmund'),
    ('Leverkusen','Bayer 04 Leverkusen'),
    ('Hoffenheim','TSG 1899 Hoffenheim'),
    ('Freiburg','SC Freiburg'),
    ('Mainz 05','1. FSV Mainz 05'),
    ('Eint Frankfurt','Eintracht Frankfurt'),
    ('Union Berlin','1. FC Union Berlin'),
    ('RB Leipzig','RB Leipzig'),
    ('Köln','1. FC Köln'),
    ('Bochum','VfL Bochum 1848'),
    ('Wolfsburg','VfL Wolfsburg'),
    ('M\'Gladbach','Borussia Mönchengladbach'),
    ('Hertha BSC','Hertha BSC'),
    ('Stuttgart','VfB Stuttgart'),
    ('Augsburg','FC Augsburg'),
    ('Arminia','DSC Arminia Bielefeld'),
    ('Greuther Fürth','SpVgg Greuther Fürth'),
    # Premier League
    ('Manchester City','Manchester City'),
    ('Liverpool','Liverpool'),
    ('Chelsea','Chelsea'),
    ('Arsenal','Arsenal'),
    ('West Ham','West Ham United'),
    ('Manchester Utd','Manchester United'),
    ('Tottenham','Tottenham Hotspur'),
    ('Wolves','Wolverhampton Wanderers'),
    ('Leicester City','Leicester City'),
    ('Aston Villa','Aston Villa'),
    ('Crystal Palace','Crystal Palace'),
    ('Brentford','Brentford'),
    ('Brighton','Brighton & Hove Albion'),
    ('Everton','Everton'),
    ('Southampton','Southampton'),
    ('Leeds United','Leeds United'),
    ('Watford','Watford'),
    ('Burnley','Burnley'),
    ('Newcastle Utd','Newcastle United'),
    ('Norwich City','Norwich City'),
    # Serie A
    ('Inter','Inter'),
    ('Milan','Milan'),
    ('Atalanta','Atalanta'),
    ('Napoli','Napoli'),
    ('Fiorentina','Fiorentina'),
    ('Juventus','Juventus'),
    ('Lazio','Lazio'),
    ('Roma','Roma'),
    ('Empoli','Empoli'),
    ('Bologna','Bologna'),
    ('Sassuolo','Sassuolo'),
    ('Hellas Verona','Hellas Verona'),
    ('Torino','Torino'),
    ('Sampdoria','Sampdoria'),
    ('Udinese','Udinese'),
    ('Venezia','Venezia FC'),
    ('Spezia','Spezia'),
    ('Genoa','Genoa'),
    ('Cagliari','Cagliari'),
    ('Salernitana','US Salernitana 1919'),
    # Ligue 1
    ('Paris S-G','Paris Saint-Germain'),
    ('Marseille','Olympique de Marseille'),
    ('Rennes','Stade Rennais FC'),
    ('Nice','OGC Nice'),
    ('Montpellier','Montpellier HSC'),
    ('Lens','Racing Club de Lens'),
    ('Strasbourg','RC Strasbourg Alsace'),
    ('Monaco','AS Monaco'),
    ('Angers','Angers SCO'),
    ('Nantes','FC Nantes'),
    ('Lille','LOSC Lille'),
    ('Brest','Stade Brestois 29'),
    ('Lyon','Olympique Lyonnais'),
    ('Reims','Stade de Reims'),
    ('Bordeaux','FC Girondins de Bordeaux'),
    ('Clermont Foot','Clermont Foot 63'),
    ('Troyes','ESTAC Troyes'),
    ('Metz','FC Metz'),
    ('Lorient','FC Lorient'),
    ('Saint-Étienne','AS Saint-Étienne'),
]

In [22]:
for team in team_names:
    df_sum = df_sum.replace(team[0], team[1])

In [23]:
def get_fifa_data(df_fifa, df_team):

    def data_check(array, value):
        if len(np.array(value)) > 0:
            array.append(np.array(value)[0])
        else:
            array.append(np.nan)

        return array

    overall_values = []
    potential_values = []

    pace_values = []
    shooting_values = []
    passing_values = []
    dribbling_values = []
    defending_values = []
    physic_values = []
    
    wage_eur_values = []
    value_eur_values = []

    base = r'^{}'
    expr = '(?=.*{})'

    for name in np.array(df_team.player):
        names = name.split(' ')
        b = base.format(''.join(expr.format(w) for w in names))

        player = df_fifa[df_fifa.long_name.str.contains(b)]
        
        overall_values = data_check(overall_values, player.overall)
        potential_values = data_check(potential_values, player.potential)
                                      
        pace_values = data_check(pace_values, player.pace)
        shooting_values = data_check(shooting_values, player.shooting)
        passing_values = data_check(passing_values, player.passing)
        dribbling_values = data_check(dribbling_values, player.dribbling)
        defending_values = data_check(defending_values, player.defending)
        physic_values = data_check(physic_values, player.physic)
        
        wage_eur_values = data_check(wage_eur_values, player.wage_eur)
        value_eur_values = data_check(value_eur_values, player.value_eur)

    df_team["overall"] = overall_values
    df_team["potential"] = potential_values

    df_team["pace"] = pace_values
    df_team["shooting"] = shooting_values
    df_team["passing"] = passing_values
    df_team["dribbling"] = dribbling_values
    df_team["defending"] = defending_values
    df_team["physic"] = physic_values 
    
    df_team["wage_eur"] = wage_eur_values 
    df_team["value_eur"] = value_eur_values 

    return df_team

In [24]:
fifa_all = get_fifa_data(fifa_data, df_sum)

In [25]:
fifa_all.columns

Index(['season', 'player', 'nationality', 'position', 'squad', 'age',
       'birth_year', 'games', 'games_starts', 'minutes',
       ...
       'overall', 'potential', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'wage_eur', 'value_eur'],
      dtype='object', length=187)

In [26]:
fifa_all

,season,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,...,overall,potential,pace,shooting,passing,dribbling,defending,physic,wage_eur,value_eur
0,2022,Aaron Connolly,ie IRL,FW,Brighton & Hove Albion,21,2000,45.0,24.0,2205.0,...,70.0,77.0,75.0,70.0,50.0,71.0,20.0,64.0,23000.0,3300000.0
1,2022,Aaron Cresswell,eng ENG,DF,West Ham United,32,1989,100.0,98.0,8626.0,...,79.0,79.0,70.0,60.0,75.0,76.0,77.0,68.0,66000.0,12500000.0
2,2022,Aaron Hickey,sct SCO,DF,Bologna,19,2002,29.0,27.0,2121.0,...,69.0,82.0,69.0,50.0,65.0,67.0,66.0,56.0,7000.0,3200000.0
3,2022,Aaron Lennon,eng ENG,MF,Burnley,34,1987,38.0,19.0,1826.0,...,73.0,73.0,78.0,60.0,70.0,77.0,48.0,56.0,28000.0,1400000.0
4,2021,Aaron Leya Iseka,be BEL,FW,FC Metz,22,1997,71.0,33.0,3229.0,...,70.0,75.0,82.0,71.0,60.0,69.0,27.0,71.0,22000.0,2400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4481,2020,Łukasz Teodorczyk,pl POL,"FW,DF",Udinese,28,1991,30.0,3.0,622.0,...,69.0,69.0,58.0,70.0,57.0,63.0,32.0,71.0,9000.0,1400000.0
4482,2022,Šime Vrsaljko,hr CRO,DF,Atlético de Madrid,29,1992,30.0,19.0,1820.0,...,80.0,80.0,79.0,55.0,73.0,74.0,78.0,77.0,56000.0,18500000.0
4483,2019,Žan Celar,si SVN,FW,Roma,19,1999,1.0,0.0,5.0,...,64.0,74.0,71.0,62.0,44.0,64.0,26.0,68.0,3000.0,1300000.0
4484,2020,Žan Majer,si SVN,MF,Lecce,27,1992,27.0,19.0,1664.0,...,67.0,67.0,61.0,58.0,69.0,70.0,55.0,65.0,2000.0,1000000.0


In [27]:
fifa_all.to_csv('top5.csv')